# Politician Activity on Facebook by Political Affiliation

The parameters in the cell below can be adjusted to explore other political affiliations and time frames.

### How to explore other political affiliation?
The ***affiliation*** parameter can be use to aggregate politicians by their political affiliations. The column `affiliation` in this [this other notebook](../politicians.ipynb?autorun=true) show the politicians that belong each political affiliation.

***Alternatively***, you can direcly use the [politicians API](http://mediamonitoring.gesis.org/api/politicians/swagger/), or access it with the [SMM Wrapper](https://pypi.org/project/smm-wrapper/).

## A. Set Up parameters

In [ ]:
# Parameters: 
affiliation = 'CSU'
from_date = '2017-09-01'
to_date = '2018-12-31'
aggregation = 'week'

## B. Using the SMM Politician API

In [ ]:
import pandas as pd

# Create an instance to the smm wrapper
from smm_wrapper import SMMPoliticians
smm = SMMPoliticians()

#using the api to get the data
df = smm.dv.get_politicians()

# Filter the accounts by party, and valid ones (the ones that contain fb_ids)
party_df = df[(df['affiliation']==affiliation) & (df['fb_ids'].notnull())] 

# query the Social Media Monitoring API
posts_by = pd.concat(smm.dv.posts_by(_id=organization_id, from_date=from_date, to_date=to_date, aggregate_by=aggregation) 
               for organization_id in party_df.index)
comments_by = pd.concat(smm.dv.comments_by(_id=organization_id, from_date=from_date, to_date=to_date, aggregate_by=aggregation) 
               for organization_id in party_df.index)

# aggregate posts and comments
total_posts_by = posts_by.groupby('date')[
    'posts', 'replies', 'shares', 'reactions', 'likes'].sum()
total_comments_by = comments_by.groupby('date')[
    'comments', 'replies', 'likes'].sum()     

## C. Plotting
### C.1 Plot Facebook Post Activity

In [ ]:
import plotly
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

#plot for facebook posts activity
plotly.offline.iplot({
    "data": [go.Scatter(x=total_posts_by.index.tolist(), y=total_posts_by['posts'], name='Posts', line_shape='spline'), 
             go.Scatter(x=total_posts_by.index.tolist(), y=total_posts_by['replies'], name='Replies',line_shape='spline'),
            go.Scatter(x=total_posts_by.index.tolist(), y=total_posts_by['shares'], name='Shares', line_shape='spline'),
            go.Scatter(x=total_posts_by.index.tolist(), y=total_posts_by['reactions'], name='Reactions', line_shape='spline'),
            go.Scatter(x=total_posts_by.index.tolist(), y=total_posts_by['likes'], name='Likes', line_shape='spline')], 
    "layout": go.Layout(title='Facebook posts for {}'.format(affiliation), xaxis={'title':''}, yaxis={'title':'N'})
})

### C.2 Plot Facebook Comment Activity

In [ ]:
# plot for facebook comments activity
plotly.offline.iplot({
    "data": [go.Scatter(x=total_comments_by.index.tolist(), y=total_comments_by['comments'], name='Comments', line_shape='spline'), 
             go.Scatter(x=total_comments_by.index.tolist(), y=total_comments_by['replies'], name='Replies', line_shape='spline'),
            go.Scatter(x=total_comments_by.index.tolist(), y=total_comments_by['likes'], name='Likes', line_shape='spline')], 
    "layout": go.Layout(title='Facebook comments for {}'.format(affiliation), xaxis={'title':''}, yaxis={'title':'N'})
})